# Understanding MEMMs

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pyemma
import mdshare

mpl.rcParams.update({'font.size': 14})

## Bias energies in multi-temperature simulations

$$b^{(i)}(\mathbf{x}) = \left( \frac{1}{\text{k}_\text{B} T^{(i)}} - \frac{1}{\text{k}_\text{B} T^{\circ}} \right) U(\mathbf{x}) = \left( \frac{1}{\text{k}_\text{B} T^{(i)}} - \frac{1}{\text{k}_\text{B} T^{\circ}} \right) \text{k}_\text{B} T^{(j)} u^{(j)}(\mathbf{x})$$

**Note**: all simulations/fragments at $T=T^\circ$ are **unbiased** while simulations/fragments at $T\neq T^\circ$ are **biased** with respect to the reference temperature.

## API functions for multi-temperature simulations

For these simulation types, the `pyemma.thermo` module provides the API function

```python
def estimate_multi_temperature(
    energy_trajs, temp_trajs, dtrajs,
    energy_unit='kcal/mol', temp_unit='K', reference_temperature=None,
    maxiter=10000, maxerr=1.0E-15, save_convergence_info=0,
    estimator='wham', lag=1, dt_traj='1 step', init=None):
    ...
```

Let us revisit the example of the asymmetric doublewell potential:

### Step 1
We start with loading the data and exploring the different trajectory types and realizations.

In [ ]:
with np.load(mdshare.fetch('pyemma-tutorial-mt-data.npz', working_directory='data')) as fh:
    trajs = [fh['conf_traj_%03d.npy' % i] for i in range(20)]
    temp_trajs = [fh['temp_traj_%03d.npy' % i] for i in range(20)]
    energy_trajs = [fh['energy_traj_%03d.npy' % i] for i in range(20)]

fig, axes = plt.subplots(len(trajs), 3, figsize=(10, 1 * len(trajs)), sharex=True)
for i, (x, t, e) in enumerate(zip(trajs, temp_trajs, energy_trajs)):
    axes[i, 0].plot(x)
    axes[i, 1].plot(t)
    axes[i, 2].plot(e)
fig.tight_layout()

### Step 2
We create a regular grid which covers the full range of all simulated trajectories and assign the latter to these grid points.

In [ ]:
x = np.linspace(np.min(trajs), np.max(trajs), 61)
centers = 0.5 * (x[:-1] + x[1:]).reshape(-1, 1)
dtrajs = pyemma.coordinates.assign_to_centers(trajs, centers=centers)

### Step 3
Applying WHAM to a *new* dataset is a good way to get a first impression.

We pass our three lists with configuration, temperature, and energy trajectories into the `estimate_multi_temperature()` API function. As both, the temperatures and energies, are given in units of $\text{k}_{\text{B}}T$, we have to pass this information into the API function, too. Finally, we choose a termination criterion and the level of convergence information output, and we make sure that the API function uses the WHAM estimator.

In [ ]:
wham = pyemma.thermo.estimate_multi_temperature(
    energy_trajs, temp_trajs, dtrajs,
    energy_unit='kT', temp_unit='kT',
    maxiter=100000, maxerr=5e-14, save_convergence_info=1,
    estimator='wham')

After the estimation process terminates, we can visualize the convergence behaviour using a convenience function from the `pyemma.plots` module.

In [ ]:
pyemma.plots.plot_convergence_info(wham);

The WHAM estimator returns the most simple multiensemble model in `pyemma`. The model contains the stationary distribution and free energy profile of the unbiased state - whether we have provided unbiased data or not:

In [ ]:
fig, (ax_pi, ax_f) = plt.subplots(1, 2, figsize=(10, 4))
ax_pi.plot(centers, wham.pi_full_state)
ax_pi.set_ylabel('$\pi(x)$')
ax_f.plot(centers, wham.f_full_state)
ax_f.set_ylabel('$f(x)$')
for ax in (ax_pi, ax_f):
    ax.set_xlabel('$x$')
fig.tight_layout()

In addition, we have a model for each of the provided thermodynamic states which all include a stationary distribution and free energy profile.

In [ ]:
fig, (ax_pi, ax_f) = plt.subplots(1, 2, figsize=(10, 4))
ax_pi.plot(centers, wham.pi_full_state, ':o', color='black', label='unbiased')
ax_pi.set_ylabel('$\pi(x)$')
ax_f.plot(centers, wham.f_full_state, ':o', color='black')
ax_f.set_ylabel('$f(x)$')
for i, model in enumerate(wham.models):
    ax_pi.plot(centers, model.pi_full_state, label='T=%.2f kT' % wham.temperatures[i])
    ax_f.plot(centers, model.f_full_state)
for ax in (ax_pi, ax_f):
    ax.set_xlabel('$x$')
ax_pi.legend()
fig.tight_layout()

We can see in the above figure that the model for $T=1\text{k}_{\text{B}}T$ yields the same result as the unbiased thermodynamic state of the WHAM estimation. This is because the `estimate_multi_temperature()` API function use, unless requested otherwise, the lowest temperature as reference and, hence, all simulations/fragments at $T=1\text{k}_{\text{B}}T$ are unbiased.

### Step 4
To obtain kinetic results from our dataset, we apply DTRAM. The call is nearly the same as for WHAM: we just have to change the `estimator` parameter to `dtram` and specify one or more lag times. If we use a single lag time, the function returns a single `MEMM` and if we give more than one lag time, the function instead returns a list of `MEMM` objects.

In [ ]:
dtram = pyemma.thermo.estimate_multi_temperature(
    energy_trajs, temp_trajs, dtrajs,
    energy_unit='kT', temp_unit='kT',
    maxiter=20000, maxerr=5e-14, save_convergence_info=20,
    estimator='dtram', lag=[i + 1 for i in range(10)])

Agan, we visualize the convergence behaviour. Now that we have estimates for several lag times, the plotting functions draws one set of convergence curves for each MEMM.

In addition, we can also visualize how the `MEMM` implied timescales (for the unbiased thermodynamic state) converge with the chosen lag times.

In [ ]:
fig, (ax_inc, ax_lli, ax_its) = plt.subplots(3, 1, figsize=(10, 15))
pyemma.plots.plot_convergence_info(dtram, axes=[ax_inc, ax_lli])
pyemma.plots.plot_memm_implied_timescales(dtram, ax=ax_its, nits=5)
fig.tight_layout()

Based on the implied timescale plot, we select an `MEMM` at a suitable lag time and continue with our analysis.

Like WHAM, an MEMM contains the unbiased stationary distribution and free energy profile as well as a list of internal models for all provided thermodynamic states.

In [ ]:
memm = dtram[2]

fig, (ax_pi, ax_f) = plt.subplots(1, 2, figsize=(10, 4))
ax_pi.plot(centers, memm.pi_full_state, ':o', color='black', label='unbiased')
ax_pi.set_ylabel('$\pi(x)$')
ax_f.plot(centers, memm.f_full_state, ':o', color='black')
ax_f.set_ylabel('$f(x)$')
for i, model in enumerate(memm.models):
    ax_pi.plot(centers, model.pi_full_state, label='T=%.2f kT' % memm.temperatures[i])
    ax_f.plot(centers, model.f_full_state)
for ax in (ax_pi, ax_f):
    ax.set_xlabel('$x$')
ax_pi.legend()
fig.tight_layout()

In contrast to WHAM, each internal model of an `MEMM` is itself a Markov state model object.

**Note**: only if we have supplied unbised data can the estimated `MEMM` provide an MSM for the unbiased state!

Now that we have an MSM, we can search for metastable states and compute, e.g., mean first passage times (MFPTs) between them.

In [ ]:
# extract the unbiased MSM
msm = memm.msm

# find metastable sets
pcca = msm.pcca(2)

# visualize the metastable sets
fig, (ax_pi, ax_f) = plt.subplots(1, 2, figsize=(10, 4))
ax_pi.plot(centers, msm.pi_full_state, '--', color='black', label='unbiased')
ax_f.plot(centers, msm.f_full_state, '--', color='black')
for i, s in enumerate(pcca.metastable_sets):
    sm = msm.active_set[s]
    ax_pi.scatter(centers[sm], msm.pi_full_state[sm], c='C%d' % i, label='state %d' % (i + 1))
    ax_f.scatter(centers[sm], msm.f_full_state[sm], c='C%d' % i)
ax_pi.legend()
ax_pi.set_ylabel('$\pi(x)$')
ax_f.set_ylabel('$f(x)$')
for ax in [ax_pi, ax_f]:
    ax.set_xlabel('$x$')
fig.tight_layout()

Please note that only the highlighted microstates have been visited in the unbiased simulations/fragments.

In [ ]:
print('MFPT[1 -> 2] = %7.1f steps' % (msm.mfpt(pcca.metastable_sets[0], pcca.metastable_sets[1])))
print('MFPT[2 -> 1] = %7.1f steps' % (msm.mfpt(pcca.metastable_sets[1], pcca.metastable_sets[0])))

## Bias energies in umbrella sampling simulations

The bias is computed via a harmonic potential based on the deviation of a frame from a reference structure. In the usual one-dimensional case, this reads

$$b^{(i)}(\mathbf{x}) = \frac{k^{(i)}}{2} \left\Vert \mathbf{x} - \mathbf{x}^{(i)} \right\Vert^2.$$

In the more general case, though, one can use a non-symmetric force matrix:

$$b^{(i)}(\mathbf{x}) = \frac{1}{2} \left\langle \mathbf{x} - \mathbf{x}^{(i)} \middle\vert \mathbf{k}^{(i)} \middle\vert \mathbf{x} - \mathbf{x}^{(i)} \right\rangle.$$

## API functions for umbrella sampling

For these simulation types, the `pyemma.thermo` module provides the API function

```python
def estimate_umbrella_sampling(
    us_trajs, us_dtrajs, us_centers, us_force_constants,
    md_trajs=None, md_dtrajs=None, kT=None,
    maxiter=10000, maxerr=1.0E-15, save_convergence_info=0,
    estimator='wham', lag=1, dt_traj='1 step', init=None):
    ...

```

### Step 1
We start by loading and visualizing the data.

In [ ]:
with np.load(mdshare.fetch('pyemma-tutorial-us-data.npz', working_directory='data')) as fh:
    # load biased data
    us_trajs = [fh['us_traj_%03d.npy' % i] for i in range(100)]
    us_centers = fh['umbrella_centers'].tolist()
    us_force_constants = fh['force_constants'].tolist()
    # load unbiased data
    md_trajs = [fh['md_traj_%03d.npy' % i] for i in range(5)]

fig, (ax_us, ax_md) = plt.subplots(1, 2, figsize=(10, 4))
for traj in us_trajs:
    ax_us.plot(traj)
for traj in md_trajs:
    ax_md.plot(traj)
for ax in [ax_us, ax_md]:
    ax.set_xlabel('steps')
    ax.set_ylabel('$x$')
fig.tight_layout()

The umbrella sampling data seems to overlap nicely (left) but the unbiased data appears to be nonreversible (right).

### Step 2
We create a regular grid which covers the full range of all simulated trajectories and assign the latter to these grid points.

In [ ]:
x = np.linspace(min(np.min(us_trajs), np.min(md_trajs)), max(np.max(us_trajs), np.max(md_trajs)), 61)
centers = 0.5 * (x[:-1] + x[1:]).reshape(-1, 1)
us_dtrajs = pyemma.coordinates.assign_to_centers(us_trajs, centers=centers)
md_dtrajs = pyemma.coordinates.assign_to_centers(md_trajs, centers=centers)

### Step 3
Apply WHAM to **only** the biased data and visualize.

In [ ]:
wham = pyemma.thermo.estimate_umbrella_sampling(
    us_trajs, us_dtrajs, us_centers, us_force_constants,
    md_trajs=None, md_dtrajs=None,
    maxiter=100000, maxerr=5e-14, save_convergence_info=1,
    estimator='wham')

fig, axes = plt.subplots(2, 2, figsize=(10, 8))
pyemma.plots.plot_convergence_info(wham, axes=axes[0, :])
axes[1, 0].plot(centers, wham.pi_full_state)
axes[1, 1].plot(centers, wham.f_full_state)
for ax in axes[1, :]:
    ax.set_xlabel('$x$')
axes[1, 0].set_ylabel('$\pi(x)$')
axes[1, 1].set_ylabel('$f(x)$')
fig.tight_layout()

### Exercise!
Apply WHAM to the biased and unbiased data. What happens?

In [ ]:
wham2 = # FIXME

fig, axes = plt.subplots(2, 2, figsize=(10, 8))
pyemma.plots.plot_convergence_info(wham2, axes=axes[0, :])
axes[1, 0].plot(centers, wham.pi_full_state, label='wham')
axes[1, 1].plot(centers, wham.f_full_state)
axes[1, 0].plot(centers, wham2.pi_full_state, label='wham2')
axes[1, 1].plot(centers, wham2.f_full_state)
for ax in axes[1, :]:
    ax.set_xlabel('$x$')
axes[1, 0].legend()
axes[1, 0].set_ylabel('$\pi(x)$')
axes[1, 1].set_ylabel('$f(x)$')
fig.tight_layout()

### Step 4
Apply DTRAM to the **only** the biased data and visualize.

In [ ]:
dtram = pyemma.thermo.estimate_umbrella_sampling(
    us_trajs, us_dtrajs, us_centers, us_force_constants,
    md_trajs=None, md_dtrajs=None,
    maxiter=20000, maxerr=5e-14, save_convergence_info=1,
    estimator='dtram', lag=[i + 1 for i in range(10)])

In [ ]:
fig, (ax_inc, ax_lli, ax_its) = plt.subplots(3, 1, figsize=(10, 15))
pyemma.plots.plot_convergence_info(dtram, axes=[ax_inc, ax_lli])
pyemma.plots.plot_memm_implied_timescales(dtram, ax=ax_its, nits=5)
fig.tight_layout()

What happend?

**Remember**: an `MEMM` wonly provides an unbiased `MSM`, if we provide unbiased data. By default, the plotting function looks for the unbiased `MSM` which leads to the observed exception.

For biased data only, the `MEMM` cannot give us unbiased kinetics. We can still plot the stationary properties, though.

In [ ]:
memm = dtram[0]

fig, (ax_pi, ax_f) = plt.subplots(1, 2, figsize=(10, 4))
ax_pi.plot(centers, memm.pi_full_state, ':o', color='black', label='unbiased')
ax_pi.set_ylabel('$\pi(x)$')
ax_f.plot(centers, memm.f_full_state, ':o', color='black')
ax_f.set_ylabel('$f(x)$')
for model in memm.models:
    ax_pi.plot(centers, model.pi_full_state)
    ax_f.plot(centers, model.f_full_state)
for ax in (ax_pi, ax_f):
    ax.set_xlabel('$x$')
fig.tight_layout()

### Exercise!
Build an `MEMM` with a suitable lag time, visualize the stationary properties, and compute MFPTs between two metastable states.

In [ ]:
# FIXME